In [1]:
import pandas as pd
data = pd.read_csv('data/task1/train/hindi-train-low', sep='\t',names=['source_word', 'final_form', 'prop'])
source_words = data['source_word'].unique()
len(data)

100

In [2]:
alphabets = set()
for source in source_words:
    for alphabet in source:
        alphabets.add(alphabet)
print(alphabets, len(alphabets))

{'घ', 'ु', 'र', 'व', 'ख', 'ा', 'ि', 'न', 'ह', 'ग', 'फ', 'ै', 'ब', 'ड', 'ल', 'छ', '्', 'त', 'ू', 'य', 'स', 'श', 'ढ', 'ो', 'प', 'अ', 'ऐ', 'थ', 'झ', 'द', 'े', 'ँ', 'क', 'ं', 'ट', 'ठ', 'ी', 'उ', 'च', ' ', 'ध', 'इ', 'म', 'ज', '़'} 45


In [3]:
mapping = dict()
for source in source_words:
    l = len(source)
    if l not in mapping.keys():
        mapping[l] = 1
        continue
    mapping[l] += 1
mapping

{4: 9, 5: 31, 6: 16, 7: 5, 8: 3, 9: 3, 10: 6, 11: 4, 12: 4, 13: 4}

In [ ]:
EPSILON = 8  # Looking at the above frequency distribution
Ci = len(alphabets)

In [ ]:
import numpy as np
# *****************Start-Generate Matrix*******************

# Function to generate character id of the form char_i_j_k
def gen_cid(char, L, R):
    """
    Returns charater id:
    char -> character
    L -> Postion of character from LEFT
    R -> Postion of character from RIGHT(negative)
    """
    return(char + '_' + str(L) + '_' + str(R))

# Generate character ids
cids = []
for c in alphabets:
    for i in range(EPSILON + 1):
        for j in range(EPSILON + 1):
            cid = gen_cid(c, i, -j)
            cids.append(cid)
print(len(cids))
assert len(cids) == Ci*pow(EPSILON + 1, 2)  # Check for the correct number of entries

In [ ]:
# Initialize Adjaceny Matrix
adj_mat = pd.DataFrame(data=np.zeros(shape=(len(cids), len(cids))), columns=cids, index=cids)

In [ ]:
def cid_exists(cid, matrix):
    """Checks if cid is present in matrix's keys"""
    return(True if cid in matrix else False)

# Note: This entire loop takes around 36 minutes for EPSILON = 16, Ci = 37 (Russian)
# ToDo: Multithreading
for source in source_words:
    s_l = len(source)
    for (i, c_1) in enumerate(source):
        cid_1 = gen_cid(c_1, i + 1, i - s_l)

        # cid_1 subset
        subset_cid_1 = [gen_cid(c_1, 0, i - s_l), gen_cid(c_1, i, 0), gen_cid(c_1, 0, 0)]

        for (j, c_2) in enumerate(source):
            cid_2 = gen_cid(c_2, j + 1, j - s_l)

            # Case 1 (1): cid_1 --> cid_2
            if cid_exists(cid_1, adj_mat) and cid_exists(cid_2, adj_mat[cid_1]):
                adj_mat[cid_1][cid_2] += 1

            # cid_2 subset
            subset_cid_2 = [gen_cid(c_2, 0, j - s_l), gen_cid(c_2, j, 0), gen_cid(c_2, 0, 0)]
            
            # Case 2 (3): cid_1 --> sub(cid_2)
            for cid in subset_cid_2:
                if cid_exists(cid_1, adj_mat) and cid_exists(cid, adj_mat[cid_1]):
                    adj_mat[cid_1][cid] += 1
                    
            
            # Case 3 (3): sub(cid_1) --> cid_2
            for cid in subset_cid_1:
                if cid_exists(cid, adj_mat) and cid_exists(cid_2, adj_mat[cid]):
                    adj_mat[cid][cid_2] += 1
            
            # Case 4 (9): sub(cid_1) --> sub(cid_2)
            for s_cid_1 in subset_cid_1:
                for s_cid_2 in subset_cid_2:
                    if cid_exists(s_cid_1, adj_mat) and cid_exists(s_cid_2, adj_mat[s_cid_1]):
                        adj_mat[s_cid_1][s_cid_2] += 1

In [ ]:
# for (i, cid) in enumerate(adj_mat):
#     vec_sum = 0.0
#     for cid2 in adj_mat[cid]:
#         print adj_mat[cid][0]
#         vec_sum += adj_mat[cid][cid2]
#     for cid2 in adj_mat[cid]:
#         adj_mat[cid][cid2] /= vec_sum

In [ ]:
# If dataframe size increases too much
# for (i, smaller_df) in enumerate(np.array_split(adj_mat, 10)):
#     smaller_df.to_pickle('Matrices/russian/russian_' + str(i) + '.p')
# adj_mat.to_pickle('Matrices/russian.p')
adj_mat

In [ ]:
(pd.to_pickle('Matrices/maltese_2_e_8.p')['b_1_0']['b_1_0'])